In [43]:
import pandas as pd
from os import path, listdir
import tensorflow as tf
from tensorflow.keras import layers, models

In [44]:
data_path = "/Users/jeevakumar/Desktop/FADML/project/CrisisMMD_v2.0/crisismmd_datasplit_all/"
train_file = "task_damage_text_img_train.tsv"
test_file = "task_damage_text_img_test.tsv"
train_data = pd.read_csv(path.join(data_path, train_file), sep='\t')
test_data = pd.read_csv(path.join(data_path, test_file), sep='\t')

In [45]:
test_data.head()

,event_name,tweet_id,image_id,tweet_text,image,label
0,hurricane_maria,912065374929264640,912065374929264640_1,἟5἟7἟5἟7἟5἟7#HurricaneMaria #huracanmaria hato...,data_image/hurricane_maria/24_9_2017/912065374...,little_or_no_damage
1,hurricane_harvey,905930890735439873,905930890735439873_1,Some of the devastation from #Harvey friends a...,data_image/hurricane_harvey/7_9_2017/905930890...,mild_damage
2,hurricane_irma,910225185176997895,910225185176997895_0,Experts say Maria could hit Puerto Rico harder...,data_image/hurricane_irma/19_9_2017/9102251851...,severe_damage
3,california_wildfires,917804456422473734,917804456422473734_0,#California schools call off all sports becaus...,data_image/california_wildfires/10_10_2017/917...,severe_damage
4,hurricane_maria,913116029156339713,913116029156339713_0,#Dominica's Prime Minister pleads for #aid: ht...,data_image/hurricane_maria/27_9_2017/913116029...,severe_damage


In [46]:
train_data.head()

,event_name,tweet_id,image_id,tweet_text,image,label
0,hurricane_harvey,905960092822003712,905960092822003712_0,"RT @tveitdal: Where Harvey is hitting hardest,...",data_image/hurricane_harvey/8_9_2017/905960092...,severe_damage
1,california_wildfires,918008272363368448,918008272363368448_0,"#NAPA #California #wildfires HORRIBLE, NO WORD...",data_image/california_wildfires/11_10_2017/918...,severe_damage
2,hurricane_irma,909396901254090752,909396901254090752_0,RT @business: Caribbean tax haven begins to bo...,data_image/hurricane_irma/17_9_2017/9093969012...,severe_damage
3,hurricane_maria,912097936200355841,912097936200355841_0,"@StormReportWX Steve, have you seen this from ...",data_image/hurricane_maria/24_9_2017/912097936...,severe_damage
4,hurricane_maria,922610290281402368,922610290281402368_1,Thanks for following! please share! https://t....,data_image/hurricane_maria/23_10_2017/92261029...,little_or_no_damage


In [47]:
#EDA
train_data.info()
lables = train_data.label.unique()
train_data.label =  pd.Categorical(train_data.label)
train_data.label = train_data.label.cat.codes
train_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2468 entries, 0 to 2467
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   event_name  2468 non-null   object
 1   tweet_id    2468 non-null   int64 
 2   image_id    2468 non-null   object
 3   tweet_text  2468 non-null   object
 4   image       2468 non-null   object
 5   label       2468 non-null   object
dtypes: int64(1), object(5)
memory usage: 115.8+ KB


,event_name,tweet_id,image_id,tweet_text,image,label
0,hurricane_harvey,905960092822003712,905960092822003712_0,"RT @tveitdal: Where Harvey is hitting hardest,...",data_image/hurricane_harvey/8_9_2017/905960092...,2
1,california_wildfires,918008272363368448,918008272363368448_0,"#NAPA #California #wildfires HORRIBLE, NO WORD...",data_image/california_wildfires/11_10_2017/918...,2
2,hurricane_irma,909396901254090752,909396901254090752_0,RT @business: Caribbean tax haven begins to bo...,data_image/hurricane_irma/17_9_2017/9093969012...,2
3,hurricane_maria,912097936200355841,912097936200355841_0,"@StormReportWX Steve, have you seen this from ...",data_image/hurricane_maria/24_9_2017/912097936...,2
4,hurricane_maria,922610290281402368,922610290281402368_1,Thanks for following! please share! https://t....,data_image/hurricane_maria/23_10_2017/92261029...,0


In [48]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224

image_path = '/Users/jeevakumar/Desktop/FADML/project/CrisisMMD_v2.0/'
train_data.image = train_data.image.apply(lambda x: path.join(image_path, x))

target = train_data.pop('label')
paths = tf.data.Dataset.from_tensor_slices((train_data.image.values, target.values))

for features, targets in paths.take(5):
  print ('Features: {}, Target: {}'.format(features, targets))

Features: b'/Users/jeevakumar/Desktop/FADML/project/CrisisMMD_v2.0/data_image/hurricane_harvey/8_9_2017/905960092822003712_0.jpg', Target: 2
Features: b'/Users/jeevakumar/Desktop/FADML/project/CrisisMMD_v2.0/data_image/california_wildfires/11_10_2017/918008272363368448_0.jpg', Target: 2
Features: b'/Users/jeevakumar/Desktop/FADML/project/CrisisMMD_v2.0/data_image/hurricane_irma/17_9_2017/909396901254090752_0.jpg', Target: 2
Features: b'/Users/jeevakumar/Desktop/FADML/project/CrisisMMD_v2.0/data_image/hurricane_maria/24_9_2017/912097936200355841_0.jpg', Target: 2
Features: b'/Users/jeevakumar/Desktop/FADML/project/CrisisMMD_v2.0/data_image/hurricane_maria/23_10_2017/922610290281402368_1.jpg', Target: 0


In [49]:
def get_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3) #color images
    img = tf.image.convert_image_dtype(img, tf.float32) 
    #convert unit8 tensor to floats in the [0,1]range
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT]) 

In [51]:
AUTOTUNE=tf.data.experimental.AUTOTUNE
images = paths.map(get_image, num_parallel_calls=AUTOTUNE)

In [2]:
import tensorflow as tf

# Define the file path and column names for the CSV file
csv_file_path = "/Users/jeevakumar/Desktop/FADML/project/CrisisMMD_v2.0/crisismmd_datasplit_all/task_damage_text_img_train.tsv"
image_col_name = "image"
label_col_name = "label"

# Define the batch size and image size
batch_size = 32
image_size = (224, 224)

# Define the number of classes
num_classes = 10

# Define a function to preprocess the images
def preprocess_image(image_path, label):
    # Load the image file
    image = tf.io.read_file(image_path)
    # Decode the image to a tensor
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize the image to the desired size
    image = tf.image.resize(image, image_size)
    # Convert the pixel values to floats in the range [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    # Convert the label to a one-hot encoded vector
    label = tf.one_hot(label, depth=num_classes)
    return image, label

# Create a TensorFlow dataset from the CSV file
dataset = tf.data.experimental.CsvDataset(csv_file_path, record_defaults=[tf.string, tf.int32], header=True,
                                          field_delim='\t', use_quote_delim=True)

# Map the preprocessing function to the dataset
dataset = dataset.map(preprocess_image)

# Shuffle and batch the dataset
dataset = dataset.shuffle(buffer_size=1000)
dataset = dataset.batch(batch_size)

# Print the first batch of elements in the dataset
for image_batch, label_batch in dataset.take(1):
    print("Image batch shape:", image_batch.shape)
    print("Label batch shape:", label_batch.shape)


InvalidArgumentError: Field 1 in record is not a valid int32: 905960092822003712 [Op:IteratorGetNext]